In [1]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Step 1: Load the local dataset
local_file_path = "Dataset/flight.csv"
data = pd.read_csv(local_file_path)

In [3]:
# Convert GHG emissions to numeric, handling any potential formatting issues
data['GHG QUANTITY (METRIC TONS CO2e)'] = pd.to_numeric(data['GHG QUANTITY (METRIC TONS CO2e)'], errors='coerce')

In [4]:
# Select only the required columns and remove any rows with missing values
features = ['LATITUDE', 'LONGITUDE', 'GHG QUANTITY (METRIC TONS CO2e)']
data_clean = data[features].copy()
data_clean = data_clean.dropna()

In [5]:
print(f"Original data shape: {data.shape}")
print(f"Clean data shape: {data_clean.shape}")

Original data shape: (7511, 13)
Clean data shape: (7511, 3)


In [6]:
# Scale the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data_clean)
data_scaled = pd.DataFrame(scaled_features, columns=data_clean.columns)

In [7]:
X = data_scaled[['LATITUDE', 'LONGITUDE']].values
y = data_scaled['GHG QUANTITY (METRIC TONS CO2e)'].values

In [8]:
# Print some statistics to verify the preprocessing
print("\nFeature Statistics:")
print(pd.DataFrame(X, columns=['LATITUDE', 'LONGITUDE']).describe())
print("\nTarget Statistics:")
print(pd.Series(y).describe())


Feature Statistics:
           LATITUDE     LONGITUDE
count  7.511000e+03  7.511000e+03
mean   5.827378e-16  6.489580e-16
std    1.000067e+00  1.000067e+00
min   -3.975259e+00 -4.942179e+00
25%   -7.428952e-01 -3.144797e-01
50%    2.469688e-02  6.602291e-02
75%    6.526559e-01  5.974452e-01
max    5.735741e+00  1.453011e+01

Target Statistics:
count    7.511000e+03
mean    -3.027209e-17
std      1.000067e+00
min     -3.538620e-01
25%     -3.211119e-01
50%     -2.860722e-01
75%     -1.635975e-01
max      1.671562e+01
dtype: float64


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [10]:
# Create DataLoader
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), 
                            torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [11]:
class CO2Model(nn.Module):
    def __init__(self):
        super(CO2Model, self).__init__()
        self.fc1 = nn.Linear(2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.2)
        self.batch_norm1 = nn.BatchNorm1d(128)
        self.batch_norm2 = nn.BatchNorm1d(64)
        self.batch_norm3 = nn.BatchNorm1d(32)
        
    def forward(self, x):
        x = torch.relu(self.batch_norm1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.batch_norm2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.batch_norm3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

In [12]:
class EmissionClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader):
        self.model = model
        self.train_loader = train_loader
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001, weight_decay=1e-5)
        
    def get_parameters(self, config):
        return [val.cpu().numpy() for val in self.model.parameters()]
    
    def set_parameters(self, parameters):
        for param, val in zip(self.model.parameters(), parameters):
            param.data = torch.tensor(val)
    
    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        total_loss = 0
        
        for epoch in range(3):  # Train for 3 epochs per round
            epoch_loss = 0
            for batch in self.train_loader:
                X_batch, y_batch = batch
                self.optimizer.zero_grad()
                predictions = self.model(X_batch)
                loss = self.criterion(predictions, y_batch)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)  # Gradient clipping
                self.optimizer.step()
                epoch_loss += loss.item()
            total_loss += epoch_loss / len(self.train_loader)
            print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(self.train_loader):.4f}")
            
        return self.get_parameters({}), len(self.train_loader.dataset), {"loss": total_loss / 3}
    
    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        with torch.no_grad():
            X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
            y_test_tensor = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
            predictions = self.model(X_test_tensor)
            loss = self.criterion(predictions, y_test_tensor)
            
            # Calculate R-squared for evaluation
            y_mean = y_test_tensor.mean()
            total_sum_squares = ((y_test_tensor - y_mean) ** 2).sum()
            residual_sum_squares = ((y_test_tensor - predictions) ** 2).sum()
            r2 = 1 - (residual_sum_squares / total_sum_squares)
            
        return float(loss), len(y_test), {"loss": float(loss), "r2": float(r2)}


In [13]:
def main():
    # Check if CUDA is available
    device = torch.device("cuda" if torch.cuda.is_available() else "mps")
    print(f"Using device: {device}")
    
    model = CO2Model()
    model = model.to(device)
    client = EmissionClient(model, train_loader)
    
    fl.client.start_client(
        server_address="0.0.0.0:8081",
        client=client
    )

In [14]:
if __name__ == "__main__":
    main()

Using device: mps


INFO :      
INFO :      Received: get_parameters message 3f0b4626-e5ef-4f02-afe2-104e87dcb7a9
ERROR :     Client raised an exception.
Traceback (most recent call last):
  File "/Users/Viku/GitHub/programming/.venv/lib/python3.12/site-packages/flwr/client/app.py", line 536, in start_client_internal
    reply_message = client_app(message=message, context=context)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Viku/GitHub/programming/.venv/lib/python3.12/site-packages/flwr/client/client_app.py", line 143, in __call__
    return self._call(message, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Viku/GitHub/programming/.venv/lib/python3.12/site-packages/flwr/client/client_app.py", line 126, in ffn
    out_message = handle_legacy_message_from_msgtype(
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Viku/GitHub/programming/.venv/lib/python3.12/site-packages/flwr/client/message_handler/message_handler.py", line 120, in han

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.